# ** Описание **

In [1]:
from __future__ import division

import base64
import csv
import gzip
import zlib

from collections import namedtuple

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
TRACE_NUM = 1000
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

def trace(items_num, trace_num=TRACE_NUM):
    if items_num % trace_num == 0: logging.info("Complete items %05d" % items_num)
        
def trace_worker(items_num, worker_id, trace_num=TRACE_NUM):
    if items_num % trace_num == 0: logging.info("Complete items %05d in worker_id %d" % (items_num, worker_id))

### Утилиты

#### Декораторы

In [3]:
def to_utf8(text):
    if isinstance(text, unicode): text = text.encode('utf8')
    return text

def convert2unicode(f):
    def tmp(text):
        if not isinstance(text, unicode): text = text.decode('utf8')
        return f(text)
    return tmp

def convert2lower(f):
    def tmp(text):        
        return f(text.lower())
    return tmp

#P.S. Декораторы могут усложнять отладку, так что от них вполне можно отказаться и воспользоваться copy-paste

### Извлечение текста из html

#### Извлечение текста при помощи встроенных модулей

In [4]:
from HTMLParser import HTMLParser
import re

###Извлечение текста из title можно вписать сюда

class TextHTMLParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self._text = []
        self._title = ""
        self._in_title = False

    def handle_data(self, data):
        text = data.strip()
        if len(text) > 0:
            text = re.sub('[ \t\r\n]+', ' ', text)
            self._text.append(text + ' ')

    def handle_starttag(self, tag, attrs):
        if tag == 'p':
            self._text.append('\n\n')
        elif tag == 'br':
            self._text.append('\n')
        elif tag == 'title':
            self._in_title = True

    def handle_startendtag(self, tag, attrs):
        if tag == 'br':
            self._text.append('\n\n')

    def text(self):
        return ''.join(self._text).strip()

@convert2unicode
def html2text_parser(text):
    parser = TextHTMLParser()
    parser.feed(text)
    return parser.text()

In [5]:
html2text = html2text_parser

#### Методы для токенизации текста

In [6]:
@convert2lower
@convert2unicode
def easy_tokenizer(text):
    word = unicode()
    for symbol in text:
        if symbol.isalnum(): word += symbol
        elif word:
            yield word
            word = unicode()
    if word: yield word

PYMORPHY_CACHE = {}
MORPH = None
#hint, чтобы установка pymorphy2 не была бы обязательной
def get_lemmatizer():
    import pymorphy2
    global MORPH
    if MORPH is None: MORPH = pymorphy2.MorphAnalyzer()
    return MORPH

@convert2lower
@convert2unicode
def pymorphy_tokenizer(text):
    global PYMORPHY_CACHE
    for word in easy_tokenizer(text):
        word_hash = hash(word)
        if word_hash not in PYMORPHY_CACHE:
            PYMORPHY_CACHE[word_hash] = get_lemmatizer().parse(word)[0].normal_form            
        yield PYMORPHY_CACHE[word_hash]

#### Основная функция, которая вызывается для преобразования html в список слов

In [7]:
def html2word(raw_html, to_text=html2text, tokenizer=easy_tokenizer):
    return tokenizer(to_text(raw_html).lower())

In [8]:
DocItem = namedtuple('DocItem', ['doc_id', 'is_spam', 'url', 'features'])
PreDocItem = namedtuple('PreDocItem', ['doc_id', 'is_spam', 'url', 'html_data'])

def load_csv(input_file_name):    
    """
    Загружаем данные и извлекаем на лету признаки
    Сам контент не сохраняется, чтобы уменьшить потребление памяти - чтобы
    можно было запускать даже на ноутбуках в классе
    """
    predocs = []
    with gzip.open(input_file_name) if input_file_name.endswith('gz') else open(input_file_name)  as input_file:            
        headers = input_file.readline()
        
        for i, line in enumerate(input_file):
            trace(i)
            parts = line.strip().split('\t')
            url_id = int(parts[0])                                        
            mark = int(parts[1])                   
            url = parts[2]
            pageInb64 = parts[3]
            html_data = base64.b64decode(pageInb64)
            predocs.append(PreDocItem(url_id, mark, url, ' '.join(html2word(html_data))))                  
        trace(i, 1)
    return predocs

In [9]:
%%time

TRAIN_DATA_FILE  = '/home/vanyadeg/dev/ifmo-infosearch-hw/lab-spam/data/kaggle_train_data_tab.csv'

train_predocs = list(load_csv(TRAIN_DATA_FILE))

22:41:06 INFO:Complete items 00000
22:41:20 INFO:Complete items 01000
22:41:31 INFO:Complete items 02000
22:41:42 INFO:Complete items 03000
22:41:52 INFO:Complete items 04000
22:42:04 INFO:Complete items 05000
22:42:16 INFO:Complete items 06000
22:42:28 INFO:Complete items 07000
22:42:28 INFO:Complete items 07043


CPU times: user 1min 22s, sys: 378 ms, total: 1min 22s
Wall time: 1min 22s


In [10]:
%%time

TEST_DATA_FILE  = '/home/vanyadeg/dev/ifmo-infosearch-hw/lab-spam/data/kaggle_test_data_tab.csv'

test_predocs = load_csv(TEST_DATA_FILE)

22:42:28 INFO:Complete items 00000
22:42:44 INFO:Complete items 01000
22:42:59 INFO:Complete items 02000
22:43:12 INFO:Complete items 03000
22:43:24 INFO:Complete items 04000
22:43:36 INFO:Complete items 05000
22:43:50 INFO:Complete items 06000
22:44:02 INFO:Complete items 07000
22:44:15 INFO:Complete items 08000
22:44:28 INFO:Complete items 09000
22:44:44 INFO:Complete items 10000
22:44:57 INFO:Complete items 11000
22:45:09 INFO:Complete items 12000
22:45:21 INFO:Complete items 13000
22:45:34 INFO:Complete items 14000
22:45:47 INFO:Complete items 15000
22:46:01 INFO:Complete items 16000
22:46:02 INFO:Complete items 16038


CPU times: user 3min 34s, sys: 808 ms, total: 3min 34s
Wall time: 3min 33s


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

def count_avg(words):
    avg_word_len = 0
    for word in words:
        avg_word_len += len(word)
    return avg_word_len / len(words)

def count_in_tag(html_data, pattern):
    res = 0
    anchors = re.findall(pattern, html_data)
    for anchor in anchors:
        res += len(list(html2word(anchor)))
    return res
    
def calc_features(predocs):
    vectorizer = TfidfVectorizer(min_df=0.01)
    X = vectorizer.fit_transform([pd.html_data for pd in predocs]).toarray()
    for i, pd in enumerate(predocs):
        features = X[i]
        yield DocItem(pd.doc_id, pd.is_spam, pd.url, features)

In [12]:
train_size = len(train_predocs)
test_size = len(test_predocs)

docs = list(calc_features(train_predocs + test_predocs))
train_docs = docs[0 : train_size]
test_docs = docs[train_size:]
print len(train_docs)
print len(test_docs)
print len(docs[0].features)

7044
16039
10332


In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier

class Classifier:
    
    def __init__(self):
        self.clf = AdaBoostClassifier(n_estimators=500, learning_rate=0.8)

    def predict(self, doc):        
        return self.clf.predict([doc.features])[0]                     
    
    def predict_all(self, docs):
        res = []
        for doc_num, doc in enumerate(docs):
            trace(doc_num)
            prediction = self.predict(doc)            
            res.append( (doc.doc_id, doc.is_spam, doc.url, prediction) )
        return res
    
    def train(self, docs): 
        self.clf.fit([d.features for d in docs], [d.is_spam for d in docs])

In [14]:
classifier = Classifier()
classifier.train(train_docs)

In [15]:
with open('my_submission.csv' , 'wb') as fout:
    writer = csv.writer(fout)
    writer.writerow(['Id','Prediction'])
    predictions = classifier.predict_all(test_docs)
    print len(predictions)
    for item in predictions:
        writer.writerow([item[0], int(item[3])])

23:00:25 INFO:Complete items 00000
23:00:50 INFO:Complete items 01000
23:01:16 INFO:Complete items 02000
23:01:42 INFO:Complete items 03000
23:02:07 INFO:Complete items 04000
23:02:33 INFO:Complete items 05000
23:02:59 INFO:Complete items 06000
23:03:25 INFO:Complete items 07000
23:03:50 INFO:Complete items 08000
23:04:16 INFO:Complete items 09000
23:04:42 INFO:Complete items 10000
23:05:07 INFO:Complete items 11000
23:05:33 INFO:Complete items 12000
23:05:59 INFO:Complete items 13000
23:06:24 INFO:Complete items 14000
23:06:50 INFO:Complete items 15000
23:07:16 INFO:Complete items 16000


16039
